In [2]:
### wIOU (weighted intersaction over union) demo

import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import os

label_dic = {"Tree":[35,142,107], "Person":[0,0,255], "Bicyle":[32,11,119], "Bar":[153,153,153], "Walk":[232,35,244], "Car":[142,0,0], "Road":[128,64,128], "Guide":[152,251,152], "SKY":[180,130,70], "Traffic":[0,220,220],"Building":[70,70,70]}
# you can add class labels in label_dic

In [4]:
def wIoU_evaluation(label_dic, gt_img, t_img, weight_map):
    # label_dic : Dictionary for class labels
    # gt_img    : Ground-truth image
    # t_img     : Predicted image
    # weighted map : weight map
    
    img_H = gt_img.shape[0]
    img_W = gt_img.shape[1]

    iou_stack = []
    for key_idx in label_dic.keys():

        GT = 0 
        TP = 0
        FP = 0 
        IoU = 0

        gt_map   = np.zeros((img_H,img_W,1)) 
        test_map = np.zeros((img_H,img_W,1)) 


        gt_idx   = gt_img == label_dic[key_idx]
        test_idx = t_img  == label_dic[key_idx]

        gt_map[gt_idx[:,:,0]] = 1
        test_map[test_idx[:,:,0]] = 1
        

        for i in range(gt_map.shape[0]):
            for j in range(gt_map.shape[1]):

                if gt_map[i,j] == 1:
                    GT += weight_map[i,j]

                if gt_map[i,j] == 1 and test_map[i,j] == 1:
                    TP += weight_map[i,j]

                elif gt_map[i,j] == 0 and test_map[i,j] == 1:
                    FP += weight_map[i,j]


        if GT == 0:
            pass
            #print('no label')

        else:
            IoU = TP/(GT+FP)
            iou_stack.append(IoU)
            #print(IoU)


    meaniou = sum(iou_stack)/len(iou_stack)
    
    return meaniou

In [5]:
def get_all_distance(img, label_dic, dist):
    
    # img : input label image
    # label_dic : labels dictionary
    # dist : L1, L2, Chess, etcs
    dist_map = np.zeros([img.shape[0],img.shape[1]])
    
    
    for key_idx in label_dic.keys():
    
        label = np.abs(img - label_dic[key_idx])
        label_one_ch = label[:,:,0] + label[:,:,1] + label[:,:,2]
       
        target_label = label_one_ch == 0
        target_label = target_label.astype(np.uint8)
        dist_transform = cv2.distanceTransform(target_label, cv2.DIST_L2, 5)
       
        dist_map += dist_transform/(np.max(dist_transform)+0.01)
        
        
    return dist_map

In [6]:
## evaluate wIOU (old version)

data_path = 'dataset'
gt_path = ['d1']
label_dic = {"Tree":[35,142,107], "Person":[0,0,255], "Bicyle":[32,11,119], "Bar":[153,153,153], "Walk":[232,35,244], "Car":[142,0,0], "Road":[128,64,128], "Guide":[152,251,152], "SKY":[180,130,70]}
dist = cv2.DIST_L2

alphas = [0.01,0.1,1,10,100]

for alpha in alphas:
    
    print('-------------')
    print(f'alpha: {alpha}')

    for gt_idx in gt_path:

        test_path = gt_idx + '_test2'

        # load GT image
        gt_img_path = os.path.join(data_path, gt_idx,'gt.png')
        gt_img = cv2.imread(gt_img_path)

        test_image_paths = os.path.join(data_path,test_path)
        test_img_names = os.listdir(test_image_paths)

        # distance map 
        dist_map = get_all_distance(gt_img, label_dic, dist)
        weight_map = np.exp(-alpha*dist_map)


        for img_name in test_img_names:

            test_img_path = os.path.join(test_image_paths,img_name)
            t_img  = cv2.imread(test_img_path)

            IoU = wIoU_evaluation(label_dic, gt_img, t_img, weight_map)
            
            print(f'wiou_evaluation for {img_name}: {IoU}')

   
print('done')

-------------
alpha: 0.01
wiou_evaluation for t0.png: 0.9720999204511794
wiou_evaluation for t1.png: 0.9721810563377018
wiou_evaluation for t2.png: 0.971965935033236
-------------
alpha: 0.1
wiou_evaluation for t0.png: 0.972654697619849
wiou_evaluation for t1.png: 0.9734433760437726
wiou_evaluation for t2.png: 0.9713320664827532
-------------
alpha: 1
wiou_evaluation for t0.png: 0.9778669750539619
wiou_evaluation for t1.png: 0.9837084357058519
wiou_evaluation for t2.png: 0.9648644990252547
-------------
alpha: 10
wiou_evaluation for t0.png: 0.9981328119601266
wiou_evaluation for t1.png: 0.9999657142494232
wiou_evaluation for t2.png: 0.9159274879323555
-------------
alpha: 100
wiou_evaluation for t0.png: 0.9999999999998652
wiou_evaluation for t1.png: 1.0
wiou_evaluation for t2.png: 0.8808716807062982
done


## NEW!
- Codes were optimizated using torch tensor (faster than the original codes)
- It requires torch

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import os
import torch  # new requirement

def get_all_distance(img, label_dic):
    img_tensor = torch.from_numpy(img).float().permute(2, 0, 1)
    dist_map = torch.zeros((img.shape[0], img.shape[1]))
    
    for color in label_dic.values():
        color_tensor = torch.tensor(color).float().view(3, 1, 1)
        label = torch.abs(img_tensor - color_tensor)
        label_one_ch = label.sum(dim=0)
        target_label = (label_one_ch == 0).float()
        
        target_np = target_label.cpu().numpy().astype(np.uint8)
        dist_transform = cv2.distanceTransform(target_np, cv2.DIST_L2, 5)
        dist_transform = torch.from_numpy(dist_transform).float()
        
        dist_map += dist_transform / (dist_transform.max() + 0.01)
    
    return dist_map

def wIoU_evaluation(label_dic, gt_img, t_img, weight_map):
    gt_tensor = torch.from_numpy(gt_img).float()
    t_tensor = torch.from_numpy(t_img).float()
    weight_tensor = torch.from_numpy(weight_map).float()
    
    iou_stack = []
    
    for color in label_dic.values():
        color_tensor = torch.tensor(color).float()
        
        gt_map = (gt_tensor == color_tensor.view(1, 1, 3)).all(dim=2).float()
        test_map = (t_tensor == color_tensor.view(1, 1, 3)).all(dim=2).float()
        
        GT = (gt_map * weight_tensor).sum()
        TP = ((gt_map * test_map) * weight_tensor).sum()
        FP = ((1 - gt_map) * test_map * weight_tensor).sum()
        
        if GT == 0:
            continue
        
        IoU = TP / (GT + FP)
        iou_stack.append(IoU.item())
    
    if len(iou_stack) == 0:
        return 0
    
    return sum(iou_stack) / len(iou_stack)

def evaluate_wiou(data_path, gt_path, label_dic, alphas):
    for gt_idx in gt_path:
        gt_img_path = os.path.join(data_path, gt_idx, 'gt.png')
        test_path = os.path.join(data_path, gt_idx + '_test2')
        
        gt_img = cv2.imread(gt_img_path)
        dist_map = get_all_distance(gt_img, label_dic)
        
        for alpha in alphas:
            print('-------------')
            print(f'alpha: {alpha}')
            
            weight_map = torch.exp(-alpha * dist_map).numpy()
            
            test_img_names = os.listdir(test_path)
            for img_name in test_img_names:
                test_img_path = os.path.join(test_path, img_name)
                t_img = cv2.imread(test_img_path)
                
                IoU = wIoU_evaluation(label_dic, gt_img, t_img, weight_map)
                print(f'wiou_evaluation: {IoU}')
    
    print('done')

# Example usage
data_path = 'dataset'
gt_path = ['d1']
label_dic = {"Tree":[35,142,107], "Person":[0,0,255], "Bicyle":[32,11,119], "Bar":[153,153,153], "Walk":[232,35,244], "Car":[142,0,0], "Road":[128,64,128], "Guide":[152,251,152], "SKY":[180,130,70]}
alphas = [0.01, 0.1, 1, 10, 100]

evaluate_wiou(data_path, gt_path, label_dic, alphas)

-------------
alpha: 0.01
wiou_evaluation: 0.9720999151468277
wiou_evaluation: 0.9721810668706894
wiou_evaluation: 0.9719659239053726
-------------
alpha: 0.1
wiou_evaluation: 0.9726547300815582
wiou_evaluation: 0.973443403840065
wiou_evaluation: 0.9713320881128311
-------------
alpha: 1
wiou_evaluation: 0.9778669625520706
wiou_evaluation: 0.9837084114551544
wiou_evaluation: 0.9648644626140594
-------------
alpha: 10
wiou_evaluation: 0.9981328099966049
wiou_evaluation: 0.999965712428093
wiou_evaluation: 0.915927454829216
-------------
alpha: 100
wiou_evaluation: 1.0
wiou_evaluation: 1.0
wiou_evaluation: 0.8808716833591461
done
